# **ASIRRA base notebook for AI for Astrophysics AAIF Doctoral Course**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Deyht/AI_astro_ED_AAIF/blob/main/practical_works/CNN/classification/ASIRRA/ASIRRA_CIANNA.ipynb)

---


**Link to the CIANNA github repository**
https://github.com/Deyht/CIANNA

### **CIANNA installation**

#### Query GPU allocation and properties

If nvidia-smi fail, it might indicate that you launched the colab session whithout GPU reservation.  
To change the type of reservation go to "Runtime"->"Change runtime type" and select "GPU" as your hardware accelerator.

In [ ]:
%%shell

nvidia-smi

cd /content/

git clone https://github.com/NVIDIA/cuda-samples/

cd /content/cuda-samples/Samples/1_Utilities/deviceQuery/

cmake CMakeLists.txt

make SMS="50 60 70 80"

./deviceQuery | grep Capability | cut -c50- > ~/cuda_infos.txt
./deviceQuery | grep "CUDA Driver Version / Runtime Version" | cut -c57- >> ~/cuda_infos.txt

cd ~/

If you are granted a GPU that supports high FP16 compute scaling (e.g the Tesla T4), it is advised to change the mixed_precision parameter in the prediction to "FP16C_FP32A".  
See the detail description on mixed precision support with CIANNA on the [Systeme Requirements](https://github.com/Deyht/CIANNA/wiki/1\)-System-Requirements) wiki page.

#### Clone CIANNA git repository

In [ ]:
%%shell

cd /content/

git clone https://github.com/Deyht/CIANNA

cd CIANNA

#### Compiling CIANNA for the allocated GPU generation

There is no guaranteed forward or backward compatibility between Nvidia GPU generation, and some capabilities are generation specific. For these reasons, CIANNA must be provided the platform GPU generation at compile time.
The following cell will automatically update all the necessary files based on the detected GPU, and compile CIANNA.

In [ ]:
%%shell

cd /content/CIANNA

mult="10"
cat ~/cuda_infos.txt
comp_cap="$(sed '1!d' ~/cuda_infos.txt)"
cuda_vers="$(sed '2!d' ~/cuda_infos.txt)"

lim="11.1"
old_arg=$(awk '{if ($1 < $2) print "-D CUDA_OLD";}' <<<"${cuda_vers} ${lim}")

sm_val=$(awk '{print $1*$2}' <<<"${mult} ${comp_cap}")

gen_val=$(awk '{if ($1 >= 80) print "-D GEN_AMPERE"; else if($1 >= 70) print "-D GEN_VOLTA";}' <<<"${sm_val}")

sed -i "s/.*arch=sm.*/\\t\tcuda_arg=\"\$cuda_arg -D CUDA -D comp_CUDA -lcublas -lcudart -arch=sm_$sm_val $old_arg $gen_val\"/g" compile.cp
sed -i "s/\/cuda-[0-9][0-9].[0-9]/\/cuda-$cuda_vers/g" compile.cp
sed -i "s/\/cuda-[0-9][0-9].[0-9]/\/cuda-$cuda_vers/g" src/python_module_setup.py

./compile.cp CUDA PY_INTERF

mv src/build/lib.linux-x86_64-* src/build/lib.linux-x86_64


**IMPORTANT NOTE**   
CIANNA is mainly used in a script fashion and was not designed to run in notebooks. Every cell code that directly invokes CIANNA functions must be run as a script to avoid possible errors.  
To do so, the cell must have the following structure.

```
%%shell

cd /content/CIANNA

python3 - <<EOF

[... your python code ...]

EOF
```

This syntax allows one to easily edit python code in the notebook while running the cell as a script. Note that all the notebook variables can not be accessed by the cell in this context.


### **ASIRRA**

The ASIRRA (Animal Species Image Recognition for Restricting Access) is a dataset that was originally used for CAPTCHA and HIP (Human Interactive Proofs).

The original dataset comprises 25000 images of variable resolution (averaging around 350x500) and is equally distributed over the two classes "Cat" and "Dog". For this exercise, we provide two reduced versions in the form of padded and resized RGB images at either 128x128 or 256x256 as two binary files. This construction is necessary so the dataset can fit into the limited amount of Colab RAM. You can download one or both sets to test the impact of the input resolution on your network. In these files, the first 12500 images are of Cats, and the next 12500 are of Dogs. The last 1024 images of each class will be excluded to form our reference test dataset (total size 2048).

#### Downloading and visualizing the data

We start by downloading and visualizing the raw data. You can get one or both of the resized versions.

In [ ]:
%%shell

cd /content

wget https://share.obspm.fr/s/6TBsCpAASeETH3S/download/asirra_bin_128.tar.gz
tar -xvzf asirra_bin_128.tar.gz

#wget https://share.obspm.fr/s/52nxyfn7PjzawSe/download/asirra_bin_256.tar.gz
#tar -xvzf asirra_bin_256.tar.gz

In [ ]:
%cd /content/

import os
import matplotlib.pyplot as plt
import numpy as np

image_size = 128

v_width = 8; v_height = 5
nb_images = v_width*v_height

f_im_s = image_size*image_size*3

subset_cats = np.reshape(np.fromfile("asirra_bin_%d.dat"%(image_size),
  dtype="uint8", count=f_im_s*(nb_images//2)), (nb_images//2,image_size,image_size,3))

subset_dogs = np.reshape(np.fromfile("asirra_bin_%d.dat"%(image_size),
  dtype="uint8", count=f_im_s*(nb_images//2), offset=12500*f_im_s), (nb_images//2,image_size,image_size,3))

fig, ax = plt.subplots(v_height, v_width, figsize=(v_width*1.5,v_height*1.5), dpi=200, constrained_layout=True)

for i in range(0, v_width*v_height):
  c_x = i // v_width; c_y = i % v_width
  p_c = int((i)%2) #Alternate cats and dogs in display
  if(p_c == 0):
    ax[c_x,c_y].imshow(subset_cats[i//2])
  else:
    ax[c_x,c_y].imshow(subset_dogs[i//2])
  ax[c_x,c_y].axis('off')

plt.show()

#### Data handling and augmentation

To ease data manipulation and hyperparameter exploration, we first provide a set of helper functions. To make them accessible within the CIANNA script cells, we need to export them to a Python file. Every time you want to change the content of these functions, you will need to rerun the cell to generate a new .py file. If loaded in an interactive cell, you will need to restart the kernel after changing this file to re-import it properly.

In [ ]:
%%writefile helper.py

import numpy as np
import matplotlib.pyplot as plt
import os, sys, gc, glob, time, cv2
from threading import Thread
from PIL import Image
import albumentations as A

def data_prep(nb_images_per_iter, raw_image_size, image_size, test_mode=0):
  #Data arrays are declared as global so we can work in place to reduce RAM footpring
  global raw_images, input_data, targets, input_val, targets_val

  raw_images = np.reshape(np.fromfile("asirra_bin_%d.dat"%(raw_image_size), dtype="uint8"), (25000, raw_image_size, raw_image_size,3))

  if(test_mode == 0):
    input_data = np.zeros((nb_images_per_iter,3*image_size**2), dtype="float32") #CIANNA expects "float32" arrays
    targets = np.zeros((nb_images_per_iter,2), dtype="float32")

  input_val = np.zeros((2048,3*image_size**2), dtype="float32")
  targets_val = np.zeros((2048,2), dtype="float32")


def create_augmented_batch(A_transform):

  nb_images = np.shape(input_data)[0]

  for i in range(0,nb_images):

    l_class = np.random.randint(0,2)
    l_id = np.random.randint(0,12500 - 1024)
    #Last 1024 iamges of each class kept only for the val/test set

    patch = raw_images[l_class*12500+l_id]
    transformed = A_transform(image=patch)
    patch_aug = transformed['image']

    image_size = np.shape(patch_aug)[0]

    #CIANNA expects data formated as 2D numpy arrays representing a list of flattened images (with every channel flattened after the others)
    for depth in range(0,3): #We normalize based on mean pixel value
      input_data[i,depth*image_size**2:(depth+1)*image_size**2] = (patch_aug[:,:,depth].flatten("C") - 100.0)/155.0

    targets[i,:] = 0.0
    targets[i,l_class] = 1.0

  return input_data, targets


def create_validation_set(A_transform):

  for i in range(0,2048):

    l_class = i // 1024

    patch = raw_images[(1+l_class)*(12500 - 1024) + i]
    transformed = A_transform(image=patch)
    patch_aug = transformed['image']

    image_size = np.shape(patch_aug)[0]

    for depth in range(0,3):
      input_val[i,depth*image_size**2:(depth+1)*image_size**2] = (patch_aug[:,:,depth].flatten("C") - 100.0)/155.0

    targets_val[i,:] = 0.0
    targets_val[i,l_class] = 1.0

  return input_val, targets_val


def free_data_helper():
  global raw_images, input_data, targets, input_val, targets_val
  del (raw_images, input_data, targets, input_val, targets_val)
  return

We can test the helper functions with a simple example and display the produced images. The next cell illustrates how we can create an augmented batch of images for training from the raw image dataset.  

Use this example to test the effect of combining different transform operations for data augmentation. You can also test the impact of the image resolution and of the position of the resize transformation in the augmentation list.

In [ ]:
from helper import *

class_text = ["cat","dog"]
raw_image_size = 128
image_size = 128

v_width = 8; v_height = 5
nb_images_per_iter = v_width*v_height

#See Albumentation documentation for a list of existing augmentations
train_transform = A.Compose([
  #Image resize can be done after all other transform to preserve as much details as possible
  #or as the fist operation so other transforms are faster
  A.Resize(image_size,image_size, interpolation=2, p=1.0),
  A.HorizontalFlip(p=0.5),
  #Affine here act more as an aspect ratio transform than a scaling variation
  A.Affine(scale=(0.3,1.3), translate_percent=(-0.3,0.3), rotate=(-10,10), interpolation=2, p=1.0),
  A.ToGray(p=0.02),
  A.ColorJitter(brightness=(0.8,1.2), contrast=(0.8,1.2), saturation=(0.8,1.2), hue=0.15, p=1.0),
  ])

val_transform = A.Compose([ #Here only a resize, but val transform could be more complex (center crop, padding, etc)
  A.Resize(image_size, image_size, interpolation=2, p=1.0)])

data_prep(nb_images_per_iter, raw_image_size, image_size)
input_data, targets = create_augmented_batch(train_transform)

fig, ax = plt.subplots(v_height, v_width, figsize=(v_width*1.5,v_height*1.5), dpi=200, constrained_layout=True)
patch = np.zeros((image_size,image_size,3), dtype="uint8")

for i in range(0, v_width*v_height):
  c_x = i // v_width; c_y = i % v_width
  #Images in the augmented input_data array are directly in the CIANNA format.
  #We need to convert them back to classical RGB for display.
  for depth in range(0,3):
    patch[:,:,depth] = np.reshape(input_data[i,depth*image_size**2:(depth+1)*image_size**2]*155 + 100,(image_size,image_size))

  ax[c_x,c_y].imshow(patch)
  ax[c_x,c_y].text(4, 10, class_text[(np.argmax(targets[i]))], c="red", fontsize=10, clip_on=True)
  ax[c_x,c_y].axis('off')

plt.show()

free_data_helper()

#### Training a network

The following cell allows to train a neural network architecture with CIANNA using dynamical augmentation through Albumentation. The architecture has been left empty as an exercice. Try to implement a woking architecture on this dataset and then try to achieve the highest possible accuracy.

*Link to the [CIANNA](https://github.com/Deyht/CIANNA) repository. You can refer to CIANNA's [WIKI page](https://github.com/Deyht/CIANNA/wiki) for a complete framework description. You can also look at the full [API documentation](https://github.com/Deyht/CIANNA/wiki/4\)-Interface-API-documentation) to add layer types that are absent from the LeNET-5 example.
The saved models are available in the "net_save" repository that is automatically created when starting network training. The default naming scheme only refers to the training iteration, so rename your saving files with comprehensive information about your model to keep track of your progress. A saved model can be uploaded to a new Colab session for inference or further training.*

In [ ]:
%%shell
cd /content/

python3 - <<EOF

from helper import *

sys.path.insert(0,glob.glob('/content/CIANNA/src/build/lib.*/')[-1])
import CIANNA as cnn

def i_ar(int_list):
  return np.array(int_list, dtype="int")

raw_image_size = 128
image_size = 128
nb_images_per_iter = 4096 #Must likely be reduced if the image size is aumgented so examples can fit in RAM


#See Albumentation documentation for a list of existing augmentations
train_transform = A.Compose([
  #Image resize can be done after all other transform to preserve as much details as possible
  #or as the fist operation so other transforms are faster
  A.Resize(image_size,image_size, interpolation=2, p=1.0),
  A.HorizontalFlip(p=0.5),
  #Affine here act more as an aspect ratio transform than a scaling variation
  A.Affine(scale=(0.3,1.3), translate_percent=(-0.3,0.3), rotate=(-10,10), interpolation=2, p=1.0),
  A.ToGray(p=0.02),
  A.ColorJitter(brightness=(0.8,1.2), contrast=(0.8,1.2), saturation=(0.8,1.2), hue=0.15, p=1.0),
  ])

val_transform = A.Compose([ #Here only a resize, but val transform could be more complex (center crop, padding, etc)
  A.Resize(image_size, image_size, interpolation=2, p=1.0)])


#This funtion allow to launch data augmentation on a separate thread.
#This way we can train on the GPU and generate new agumented examples in parallel.
def data_augm():
  input_data, targets = create_augmented_batch(train_transform)
  cnn.delete_dataset("TRAIN_buf", silent=1)
  cnn.create_dataset("TRAIN_buf", nb_images_per_iter, input_data[:,:], targets[:,:], silent=1)
  return

#In case the creation of new augmented data is too long compared to training, you can
#increase the number of training iteration over a single augmentation
nb_iter_per_augm = 2
if(nb_iter_per_augm > 1):
  shuffle_frequency = 1
else:
  shuffle_frequency = 0


total_iter = 2000 #Should be increased with the complexity of the network and task
load_iter = 0 #Used to reload a model at a given iteration

if (len(sys.argv) > 1):
  load_iter = int(sys.argv[1])

start_iter = int(load_iter / nb_iter_per_augm)

cnn.init(in_dim=i_ar([image_size,image_size]), in_nb_ch=3, out_dim=2,
  bias=0.1, b_size=16, comp_meth='C_CUDA', dynamic_load=1,
  mixed_precision="FP32C_FP32A", adv_size=30)

data_prep(nb_images_per_iter, raw_image_size, image_size)

input_val, targets_val = create_validation_set(val_transform)
cnn.create_dataset("VALID", 2048, input_val[:,:], targets_val[:,:])
cnn.create_dataset("TEST", 2048, input_val[:,:], targets_val[:,:])
del (input_val, targets_val) #The python arrays are no longer required after import in CIANNA
gc.collect()

#Create fist augmentation before parallelization
input_data, targets = create_augmented_batch(train_transform)
cnn.create_dataset("TRAIN", nb_images_per_iter, input_data[:,:], targets[:,:])

if(load_iter > 0):
  cnn.load("net_save/net0_s%04d.dat"%load_iter, load_iter, bin=1)
else:
  #Network backbone architecture
  cnn.conv(f_size=i_ar([3,3]), nb_filters=16  , padding=i_ar([1,1]), activation="LIN")
  cnn.pool(p_size=i_ar([2,2]), p_type="MAX")
  cnn.norm(group_size=2, activation="RELU")

  cnn.conv(f_size=i_ar([3,3]), nb_filters=32  , padding=i_ar([1,1]), activation="LIN")
  cnn.pool(p_size=i_ar([2,2]), p_type="MAX")
  cnn.norm(group_size=2, activation="RELU")

  cnn.conv(f_size=i_ar([3,3]), nb_filters=64	, padding=i_ar([1,1]), activation="LIN")
  cnn.pool(p_size=i_ar([2,2]), p_type="MAX")
  cnn.norm(group_size=4, activation="RELU")

  cnn.conv(f_size=i_ar([3,3]), nb_filters=128 , padding=i_ar([1,1]), activation="RELU")
  cnn.conv(f_size=i_ar([1,1]), nb_filters=64  , padding=i_ar([0,0]), activation="RELU")
  cnn.conv(f_size=i_ar([3,3]), nb_filters=128 , padding=i_ar([1,1]), activation="LIN")
  cnn.pool(p_size=i_ar([2,2]), p_type="MAX")
  cnn.norm(group_size=4, activation="RELU")

  cnn.conv(f_size=i_ar([3,3]), nb_filters=192 , padding=i_ar([1,1]), activation="RELU")
  cnn.conv(f_size=i_ar([1,1]), nb_filters=96  , padding=i_ar([0,0]), activation="RELU")
  cnn.conv(f_size=i_ar([3,3]), nb_filters=192 , padding=i_ar([1,1]), activation="LIN")
  cnn.pool(p_size=i_ar([2,2]), p_type="MAX")
  cnn.norm(group_size=8, activation="RELU")

  cnn.conv(f_size=i_ar([1,1]), nb_filters=2 , padding=i_ar([0,0]), activation="LIN")
  cnn.pool(p_size=i_ar([1,1]), p_type="AVG", p_global=1, activation="SMAX")


cnn.print_arch_tex("./arch/", "arch", activation=1, dropout=1)

for run_iter in range(start_iter,int(total_iter/nb_iter_per_augm)):

  t = Thread(target=data_augm)
  t.start()

  cnn.train(nb_iter=nb_iter_per_augm, learning_rate=0.002, end_learning_rate=0.00003, shuffle_every=shuffle_frequency ,\
        control_interv=20, confmat=1, momentum=0.9, lr_decay=0.0012, weight_decay=0.001, save_every=20,\
        silent=0, save_bin=1, TC_scale_factor=256.0)

  if(run_iter == start_iter):
    cnn.perf_eval()

  t.join()
  cnn.swap_data_buffers("TRAIN")

EOF


#### Evaluate your model

The following cell evaluates the accuracy and inference time over the test set.

Colab usually puts the GPU into sleep mode after idling for a few seconds. Always run this cell a few times in a row to get the real execution time.



In [ ]:

%%shell

cd /content/

python3 - <<EOF

from helper import *

#Comment to access system wide install
sys.path.insert(0,glob.glob('/content/CIANNA/src/build/lib.*/')[-1])
import CIANNA as cnn

def i_ar(int_list):
  return np.array(int_list, dtype="int")

raw_image_size = 128
image_size = 128

val_transform = A.Compose([ #Here only a resize, but val transform could be more complex (center crop, padding, etc)
  A.Resize(image_size, image_size, interpolation=2, p=1.0)])

cnn.init(in_dim=i_ar([image_size,image_size]), in_nb_ch=3, out_dim=2,
  bias=0.1, b_size=512, comp_meth='C_CUDA', dynamic_load=1,
  mixed_precision="FP16C_FP32A", adv_size=30, inference_only=1)

data_prep(0, raw_image_size, image_size, test_mode=1)

#Compute on only half the validation set to reduce memory footprint
input_test, targets_test = create_validation_set(val_transform)
cnn.create_dataset("TEST", 2048, input_test[:,:], targets_test[:,:])

#Change epochs to load based on your own training
load_epoch = 0
if(load_epoch == 0):
  #You can also download a pretrained model with the same architecture
  os.system("wget https://share.obspm.fr/s/dyrJikKH2A7Sxso/download/arch2_res128_err1.76_ms89.dat")
  cnn.load("arch2_res128_err1.76_ms89.dat", load_epoch, bin=1)
else:
  cnn.load("net_save/net0_s%04d.dat"%load_epoch, load_epoch, bin=1)



#Run forward a first time to wake up the GPU and save the result
cnn.forward(repeat=1, no_error=1, saving=2, drop_mode="AVG_MODEL")

start = time.perf_counter()
#Run forward to evaluate raw model performance
cnn.forward(no_error=1, saving=0, drop_mode="AVG_MODEL")
end = time.perf_counter()

cnn.perf_eval()
compute_time = (end-start)*1000 #in miliseconds
print ("Inference time: %f ms (%d ips)"%(compute_time, int(2048/compute_time)))


pred_results = np.fromfile("fwd_res/net0_%04d.dat"%(load_epoch), dtype="float32")
pred_result = np.reshape(pred_results, (2048,-1))

pred_correct = np.shape(np.where(np.argmax(pred_result[:,:2], axis=1) == np.argmax(targets_test[:,:], axis=1)))[1]
pred_accuracy = (pred_correct/2048)*100.0
print ("Accuracy: %f, Error rate: %f"%(pred_accuracy, 100.0-pred_accuracy))

EOF